In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 8.3 MB/s 
     |████████████████████████████████| 6.6 MB 67.3 MB/s 
     |████████████████████████████████| 84 kB 3.8 MB/s 
     |████████████████████████████████| 596 kB 51.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 9.0 MB/s 
     |████████████████████████████████| 86 kB 7.0 MB/s 
     |████████████████████████████████| 1.1 MB 52.4 MB/s 
     |████████████████████████████████| 140 kB 71.1 MB/s 
     |████████████████████████████████| 212 kB 69.6 MB/s 
     |████████████████████████████████| 127 kB 68.8 MB/s 
     |████████████████████████████████| 144 kB 28.0 MB/s 
     |████████████████████████████████| 94 kB 4.5 MB/s 
     |████████████████████████████████| 271 kB 52.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.5.1
    Uninstalling dill-0.3.5.1:
      Successfully uninstalled dill-0.3.5.1
ERROR: pip's dependency resolver does not currently take into account all t

In [ ]:
from datasets import load_dataset
import pandas as pd
dataset = load_dataset("persiannlp/parsinlu_entailment")
valid=pd.DataFrame(dataset['validation'])
train=pd.DataFrame(dataset['train'])
test=pd.DataFrame(dataset['test'])

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/755 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1675 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/270 [00:00<?, ? examples/s]

Dataset parsinlu_reading_comprehension downloaded and prepared to /root/.cache/huggingface/datasets/persiannlp___parsinlu_reading_comprehension/parsinlu-repo/1.0.0/a99ad431d1637751f03fa9da3de66a41e93983d11116a0918a6efaaa72343d2d. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
train['conc'] = [(train.iloc[i]['sent1']+'<SEP>'+train.iloc[i]['sent2']) for i in range(len(train))]
test['conc'] = [(test.iloc[i]['sent1']+'<SEP>'+test.iloc[i]['sent2']) for i in range(len(test))]
valid['conc'] = [(valid.iloc[i]['sent1']+'<SEP>'+valid.iloc[i]['sent2']) for i in range(len(valid))]

In [ ]:
new_train = pd.DataFrame()
new_test = pd.DataFrame()
new_valid = pd.DataFrame()
conc = []
label = []
for i in range(len(train)):
  if train.iloc[i]['label'] in ['e','c','n']:
    conc.append(train.iloc[i]['conc'])
    label.append(train.iloc[i]['label'])
new_train['conc'] = conc
new_train['label'] = label
conc = []
label = []
for i in range(len(test)):
  if test.iloc[i]['label'] in ['e','c','n']:
    conc.append(test.iloc[i]['conc'])
    label.append(test.iloc[i]['label'])
new_test['conc'] = conc
new_test['label'] = label
conc = []
label = []
for i in range(len(valid)):
  if valid.iloc[i]['label'] in ['e','c','n']:
    conc.append(valid.iloc[i]['conc'])
    label.append(valid.iloc[i]['label'])
new_valid['conc'] = conc
new_valid['label'] = label
train = new_train
test = new_test
valid = new_valid

In [ ]:
encoded_dict = {'c':0, 'n':1, 'e':2}
train['label'] = train.label.map(encoded_dict)
test['label'] = test.label.map(encoded_dict)
valid['label'] = valid.label.map(encoded_dict)

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(train['label'])
y_test = to_categorical(test['label'])
y_valid = to_categorical(valid['label'])

In [ ]:
import transformers

In [ ]:
from transformers import AutoConfig, AutoTokenizer, TFAutoModel

config = AutoConfig.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
model = TFAutoModel.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")

Downloading:   0%|          | 0.00/434 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/919M [00:00<?, ?B/s]

Some layers from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
x_train = tokenizer(
    text=train.conc.tolist(),
    add_special_tokens=True,
    max_length=119,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_test = tokenizer(
    text=test.conc.tolist(),
    add_special_tokens=True,
    max_length=119,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_valid = tokenizer(
    text=valid.conc.tolist(),
    add_special_tokens=True,
    max_length=128,
    truncation=False,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2302: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "


In [ ]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.layers import Input, Dense

In [ ]:
max_len = 119
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = model(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(119, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(3,activation = 'softmax')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [ ]:
optimizer = Adam(
    learning_rate=3e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0
    )

loss = CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),

model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [ ]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = ({'input_ids':x_valid['input_ids'],'attention_mask':x_valid['attention_mask']}, y_valid),
    epochs=10,
    batch_size=32
)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


24/24 [==============================] - 47s 932ms/step - loss: 1.2586 - balanced_accuracy: 0.3660 - val_loss: 1.2094 - val_balanced_accuracy: 0.3556
Epoch 2/10
24/24 [==============================] - 20s 819ms/step - loss: 1.0084 - balanced_accuracy: 0.5040 - val_loss: 1.0847 - val_balanced_accuracy: 0.3852
Epoch 3/10
24/24 [==============================] - 20s 834ms/step - loss: 0.8706 - balanced_accuracy: 0.6194 - val_loss: 1.1247 - val_balanced_accuracy: 0.3889
Epoch 4/10
24/24 [==============================] - 20s 852ms/step - loss: 0.7139 - balanced_accuracy: 0.7347 - val_loss: 1.1684 - val_balanced_accuracy: 0.3963
Epoch 5/10
24/24 [==============================] - 21s 866ms/step - loss: 0.6001 - balanced_accuracy: 0.8103 - val_loss: 1.1628 - val_balanced_accuracy: 0.3889
Epoch 6/10
24/24 [==============================] - 21s 881ms/step - loss: 0.4897 - balanced_accuracy: 0.8329 - val_loss: 1.2818 - val_balanced_accuracy: 0.3852
Epoch 7/10
24/24 [===========================

In [ ]:
import numpy as np
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
y_predicted = np.argmax(predicted_raw, axis = 1)
y_true = test.label
from sklearn.metrics import classification_report
print(classification_report(y_true, y_predicted))

              precision    recall  f1-score   support

           0       0.34      0.17      0.22       561
           1       0.49      0.43      0.46       502
           2       0.45      0.71      0.55       610

    accuracy                           0.44      1673
   macro avg       0.43      0.44      0.41      1673
weighted avg       0.43      0.44      0.41      1673

